In [1]:
import os
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import SelfReflectionChain

In [6]:
llm = AzureChatOpenAI(
    deployment_name="GPT35Turbo",
    openai_api_base=os.getenv("OPENAI_API_BASE"),
    openai_api_version="2023-03-15-preview",
    openai_api_key=os.getenv("OPENAI_API_KEY"),
    openai_api_type="azure",
    temperature=0,
)

In [10]:
reflect = SelfReflectionChain(llm=llm, return_intermediate_steps=True)

In [11]:
reflect({"question": "Solve for x in 2x+2=13.5"})

{'question': 'Solve for x in 2x+2=13.5',
 'text': 'To solve for x in 2x+2=13.5, we need to isolate the variable on one side of the equation. We can do this by subtracting 2 from both sides:\n\n2x + 2 - 2 = 13.5 - 2\n\n2x = 11.5\n\nNext, we can solve for x by dividing both sides by 2:\n\n2x/2 = 11.5/2\n\nx = 5.75\n\nTherefore, x = 5.75 is the solution to the equation 2x+2=13.5.',
 'intermediate_steps': {'agent_hypothetical_response': 'x = 5.25',
  'agent_reflection_response': 'The response provided is correct, but it would be helpful to show the steps taken to arrive at the solution. This would assist the user in understanding the process of solving equations. Additionally, it would be beneficial to include a brief explanation of the steps taken, such as isolating the variable by subtracting 2 from both sides of the equation and then dividing by 2. \n\nA revised response could be: \n\nTo solve for x in 2x+2=13.5, we first need to isolate the variable on one side of the equation. We can 

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [3]:
with open("../../state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

embeddings = OpenAIEmbeddings(deployment="EmbeddingAda002", model="text-embedding-ada-002", chunk_size=1)

In [4]:
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

Using embedded DuckDB without persistence: data will be transient


In [7]:
reflect_document_qa = SelfReflectionChain(llm=llm, return_intermediate_steps=True, input_variables=["question", "document"], template="My Question is: {question}. Please use the following context: {document}")

In [8]:
query = "What did the president say about Justice Breyer"
docs = docsearch.get_relevant_documents(query)

In [9]:
reflect_document_qa(inputs={"question": query, "document": [doc.page_content for doc in docs]})

{'question': 'What did the president say about Justice Breyer',
 'document': ['Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.',
  'And for our LGBTQ+ Americans, let’s finally get the bipartisan Equality Act to my desk. The onslaught of state laws target